In [2]:
import pandas as pd
import json
import requests
from pandas import json_normalize
import numpy as np
from semanticscholar import SemanticScholar
sch = SemanticScholar()
import csv
import time
import heapq
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
import time

### Import Data

In [3]:
df = pd.read_csv('final.csv')

### Pull Spector 1 Embeddings

In [8]:
df['DOI'] = df.apply(lambda x: json.loads(x['externalids'])['DOI'], axis = 1)
id_list = df['DOI'].to_list()

i = 0

while i < 1000000:
    r = requests.post(
            'https://api.semanticscholar.org/graph/v1/paper/batch',
            params={'fields': 'abstract,embedding'},
            json={"ids": id_list[i:i + 490]}
        )

    i = i + 490

    df_json = json.dumps(r.json(), indent=2)
    test_df = json_normalize(json.loads(df_json))
    test_df = test_df[~test_df['abstract'].isnull()]
    test_df.to_csv('papers_abstracts.csv', mode ='a', header=False)

    time.sleep(5)
        
    print (i)


490
980
1470
1960
2450
2940
3430
3920
4410
4900
5390
5880
6370
6860
7350
7840
8330
8820
9310
9800
10290
10780
11270
11760
12250
12740
13230
13720
14210
14700
15190
15680
16170
16660
17150
17640
18130
18620
19110
19600
20090
20580
21070
21560
22050
22540
23030
23520
24010
24500
24990
25480
25970
26460
26950
27440
27930
28420
28910
29400
29890
30380
30870
31360
31850
32340
32830
33320
33810
34300
34790
35280
35770
36260
36750
37240
37730
38220
38710
39200
39690
40180
40670
41160
41650
42140
42630
43120
43610
44100
44590
45080
45570
46060
46550
47040
47530
48020
48510
49000
49490
49980
50470
50960
51450
51940
52430
52920
53410
53900
54390
54880
55370
55860
56350
56840
57330
57820
58310
58800
59290
59780
60270
60760
61250
61740
62230
62720
63210
63700
64190
64680
65170
65660
66150
66640
67130
67620
68110
68600
69090
69580
70070
70560
71050
71540
72030
72520
73010
73500
73990
74480
74970
75460
75950
76440
76930
77420
77910
78400
78890
79380
79870
80360
80850
81340
81830
82320
82810
83300
83

KeyboardInterrupt: 

### Format Embeddings Data

In [52]:
mapping = {df_embed.columns[1]: 'id_1', df_embed.columns[2]: 'abstract', df_embed.columns[3]: 'model', df_embed.columns[4]: 'embedding'}
df_embed = df_embed.rename(columns=mapping)
df_embed = df_embed.drop_duplicates(subset = 'abstract', keep = 'last')
df_embed = df_embed.reset_index()
df_embed.to_csv('abstracts_embedding')
print (df_embed.head(5))
print (len(df_embed))

   level_0  index   1                                      id_1  \
0        0      0   2  c856627242a754d2d756b32843523e6d7a089148   
3        3      3   7  9b9659a4e9a4a48d7c532c76dd14ee9ccd696025   
4        4      4   8  314f8d8aba804f3ba77ac167161e73950f406332   
5        5      5  15  76796fdb8de6a4625a865fca751ee1ceb4989502   
6        6      6  17  78c025475dd2a940b0ebdc3be885ad1a01fde2ca   

                                            abstract           model  \
0  Summary: The current work characterizes young ...  specter@v0.1.1   
3  Background The Brain Injury Guidelines provide...  specter@v0.1.1   
4  A 66-year-old woman presented with pustular le...  specter@v0.1.1   
5  Abstract Background Danon disease is an X-link...  specter@v0.1.1   
6  BACKGROUND Biologics are often required for tr...  specter@v0.1.1   

                                           embedding  
0  [-3.783419132232666, -2.987468957901001, -3.34...  
3  [-5.099650859832764, -5.582947731018066, 0.164...  

In [10]:
df_embed = pd.read_csv('abstracts_embedding')

In [11]:
df_embed['embedding array'] = df_embed.apply(lambda row : np.array(row['embedding'].strip('][').split(', '), dtype = float).reshape(1,-1), axis = 1)
df_embed = df_embed.reset_index(drop = True)
print (type(df_embed['embedding'][0]))
print (type(df_embed['embedding array'][0]))
print (df_embed.head(5))

<class 'str'>
<class 'numpy.ndarray'>
   Unnamed: 0  level_0  index   1                                      id_1  \
0           0        0      0   2  c856627242a754d2d756b32843523e6d7a089148   
1           3        3      3   7  9b9659a4e9a4a48d7c532c76dd14ee9ccd696025   
2           4        4      4   8  314f8d8aba804f3ba77ac167161e73950f406332   
3           5        5      5  15  76796fdb8de6a4625a865fca751ee1ceb4989502   
4           6        6      6  17  78c025475dd2a940b0ebdc3be885ad1a01fde2ca   

                                            abstract           model  \
0  Summary: The current work characterizes young ...  specter@v0.1.1   
1  Background The Brain Injury Guidelines provide...  specter@v0.1.1   
2  A 66-year-old woman presented with pustular le...  specter@v0.1.1   
3  Abstract Background Danon disease is an X-link...  specter@v0.1.1   
4  BACKGROUND Biologics are often required for tr...  specter@v0.1.1   

                                           embedding  


### Calc Similarity of Embeddings Example 1

In [12]:
example = df_embed['embedding array'][80]
cosin = []

for i in range(0,len(df_embed)):
    try: 
        res = df_embed['embedding array'][i]
        val_out = 1 - cdist(res, example, 'cosine')
        cosin.append(val_out)
    except:
        print (i)
        cosin.append(0)

### Confirm calc by checking max similarity

In [13]:
max_value = max(cosin)
max_index = cosin.index(max_value)

print (df_embed['abstract'][80] == df_embed.loc[max_index]['abstract'])
print (len(cosin))

True
240151


### Display Top 5 similarity ranking

In [15]:
top_5 = heapq.nlargest(100, range(len(cosin)), key=cosin.__getitem__)

max_index = cosin.index(max_value)

abstract = []
for i in top_5:
    print (cosin[i])
    abstract.append(df_embed.loc[i]['abstract'])

[[1.]]
[[0.81100269]]
[[0.80840463]]
[[0.8025388]]
[[0.8020896]]
[[0.80160081]]
[[0.80029626]]
[[0.80012671]]
[[0.79925581]]
[[0.79634739]]
[[0.79417126]]
[[0.79398333]]
[[0.79386986]]
[[0.79366036]]
[[0.78924773]]
[[0.78813912]]
[[0.78767597]]
[[0.78288072]]
[[0.78204333]]
[[0.78175017]]
[[0.78004856]]
[[0.77966376]]
[[0.77931865]]
[[0.77758856]]
[[0.77748765]]
[[0.77682937]]
[[0.77643427]]
[[0.77349939]]
[[0.77297556]]
[[0.77267169]]
[[0.77251627]]
[[0.77238161]]
[[0.77149863]]
[[0.77116522]]
[[0.7708249]]
[[0.77055611]]
[[0.77042392]]
[[0.77038357]]
[[0.76987788]]
[[0.7691085]]
[[0.76897641]]
[[0.76868324]]
[[0.76850764]]
[[0.76715418]]
[[0.76685403]]
[[0.76679032]]
[[0.76664837]]
[[0.7665276]]
[[0.7660499]]
[[0.76588579]]
[[0.76573136]]
[[0.76403629]]
[[0.76383959]]
[[0.76285659]]
[[0.76191269]]
[[0.76066767]]
[[0.76047025]]
[[0.76042579]]
[[0.76029325]]
[[0.76010876]]
[[0.75870135]]
[[0.7575599]]
[[0.75731395]]
[[0.7572666]]
[[0.75676671]]
[[0.75660028]]
[[0.75606981]]
[[0.7560321

In [19]:
df

,0
0,This study aims to investigate the influence o...
1,Due to the advantages of charged particles com...
2,"The successful treatment of cancer, including ..."
3,The efficacy of conventional radiation therapy...
4,Purpose This study was designed to evaluate th...
...,...
95,We previously reported frequent loss of microR...
96,"Background/Aim: Anti-apoptotic proteins, inclu..."
97,"Changes in p53 expression, apoptosis and cell ..."
98,"Radiotherapy (RT), the major anti-cancer modal..."


### Calc Similarity of Embedding Example 2, Check Calcs, and Time Function

In [79]:
start = time.time()

example = df_embed['embedding array'][9000]
cosin = []

for i in range(0,len(df_embed)):
    try: 
        res = df_embed['embedding array'][i]
        val_out = 1 - cdist(res, example, 'cosine')
        cosin.append(val_out)
    except:
        print (i)
        cosin.append(0)
        
max_value = max(cosin)
max_index = cosin.index(max_value)


print (df_embed['abstract'][9000] == df_embed.loc[max_index]['abstract'])
print (len(cosin))

top_5 = heapq.nlargest(6, range(len(cosin)), key=cosin.__getitem__)

max_index = cosin.index(max_value)

for i in top_5:
    print (cosin[i])
    print (df_embed.loc[i]['abstract'])
    
print (time.time() - start)

True
240151
[[1.]]
Amplicon sequencing generates chimeric reads which can cause spurious inferences of biological variation. I describe UCHIME2, an update of the popular UCHIME chimera detection algorithm with new modes optimized for high-resolution biological sequence reconstruction (“denoising”) and other applications. I show that chimera frequency correlates inversely with divergence, that error-free chimera prediction from sequence is impossible in principle, and that UCHIME2 achieves higher detection accuracy than previous methods.
[[0.73703842]]
Motivation Advances in the sequencing of uncultured environmental samples, dubbed metagenomics, raise a growing need for accurate taxonomic assignment. Accurate identification of organisms present within a community is essential to understanding even the most elementary ecosystems. However, current high-throughput sequencing technologies generate short reads which partially cover full-length marker genes and this poses difficult bioinform

### Calc Similarity of Embedding Example 3, Check Calcs, Time function

In [80]:
start = time.time()

example = df_embed['embedding array'][50000]
cosin = []

for i in range(0,len(df_embed)):
    try: 
        res = df_embed['embedding array'][i]
        val_out = 1 - cdist(res, example, 'cosine')
        cosin.append(val_out)
    except:
        print (i)
        cosin.append(0)
        
max_value = max(cosin)
max_index = cosin.index(max_value)


print (df_embed['abstract'][50000] == df_embed.loc[max_index]['abstract'])
print (len(cosin))

top_5 = heapq.nlargest(6, range(len(cosin)), key=cosin.__getitem__)

max_index = cosin.index(max_value)

for i in top_5:
    print (cosin[i])
    print (df_embed.loc[i]['abstract'])
    
print (time.time() - start)

True
240151
[[1.]]
Studies were carried out to compare the early morphologic changes in the cecal mucosa of mice either infected with Serpulina hyodysenteriae or exposed to the β-hemolysin of S. hyodysenteriae. Sixty-five 12–24-week-old C3H/HeOuJ mice were infected with S. hyodysenteriae by gastric intubation. Two mice were necropsied every hour for 30 hours following infection. S. hyodysenteriae was isolated from the cecal contents of each mouse at all time points. Macroscopic lesions were first apparent at 14 hours postinfection (PI), and light microscopic lesions were first apparent at 10 hours PI, earlier than has been previously reported. Ultrastructural changes, first evident at 6 hours PI, included disarray and loss of microvilli and terminal web, with dilatation of intercellular spaces. Luminal bacteria were translocated through epithelial cells to the lamina propria, where capillaries exhibited changes indicative of increased permeability. In another experiment, solutions cont

### Calc Similarity of Embedding Example 4, Check Calcs, Time function

In [81]:
start = time.time()

example = df_embed['embedding array'][200000]
cosin = []

for i in range(0,len(df_embed)):
    try: 
        res = df_embed['embedding array'][i]
        val_out = 1 - cdist(res, example, 'cosine')
        cosin.append(val_out)
    except:
        print (i)
        cosin.append(0)
        
max_value = max(cosin)
max_index = cosin.index(max_value)


print (df_embed['abstract'][200000] == df_embed.loc[max_index]['abstract'])
print (len(cosin))

top_5 = heapq.nlargest(6, range(len(cosin)), key=cosin.__getitem__)

max_index = cosin.index(max_value)

for i in top_5:
    print (cosin[i])
    print (df_embed.loc[i]['abstract'])
    
print (time.time() - start)

True
240151
[[1.]]
Black radish (Raphanus sativus L. var. niger) root has been used in folk medicine since antiquity as a natural drug for the stimulation of bile function. According to in vitro studies the squeezed juice from black radish root exhibited significant antioxidant properties. In the present study, the beneficial effect of the black radish juice on some free radical reactions in rats fed with a diet rich in lipids (20% sunflower oil, 2% cholesterol, 0.5% cholic acid in normal chow) was examined. Thiobarbituric acid reactive substances and conjugated diene concentrations were significantly higher, while the antioxidant enzyme activities and the free radical scavenging capacity were lower in hyperlipidaemic rats compared with normal controls. Supplementation of the lipid‐rich diet with black radish juice resulted in a significant improvement of the parameters mentioned above. Although the exact mechanism of the biologically active compounds in black radish on the lipid metab

### Embed user input

In [ ]:
from transformers import AutoTokenizer, AutoModel

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
model = AutoModel.from_pretrained('allenai/specter')

papers = [{'title': 'BERT', 'abstract': 'We introduce a new language representation model called BERT'},
          {'title': 'Attention is all you need', 'abstract': ' The dominant sequence transduction models are based on complex recurrent or convolutional neural networks'}]

# concatenate title and abstract
title_abs = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for d in papers]
# preprocess the input
inputs = tokenizer(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
result = model(**inputs)
# take the first token in the batch as the embedding
embeddings = result.last_hidden_state[:, 0, :]